# Read the input Data

In [158]:
import pandas as pd
import numpy as np
import math
import copy as cp
import sys
import pickle
train_raw = pd.read_csv("./concrete/train.csv", header=None).values
cols = train_raw.shape[1]
rows = train_raw.shape[0]
train_x = np.copy(train_raw)
train_x[:,cols - 1] = 1
train_y = train_raw[:, cols - 1]

test_raw = pd.read_csv("./concrete/test.csv", header=None).values
cols = test_raw.shape[1]
rows = test_raw.shape[0]
test_x = np.copy(test_raw)
test_x[:,cols - 1] = 1
test_y = test_raw[:, cols - 1]

In [159]:
print(np.reshape(train_y, (-1,1)).shape)

(53, 1)


# Batched LMS Implementation

In [198]:
def Gradien_Decent(x,y):
    m = x.shape[0]
    n = x.shape[1]
    Error = 1
    threshold = 10e-6
    Max_iter = 10000
    LearningRate = 0.01
    w = np.zeros([n,1])
    Costs = []
    iter = 0
    while Error > threshold and iter < Max_iter:
        iter = iter + 1
        diff = np.matmul(x, w) - np.reshape(y, (-1,1))      
        gradient = np.sum(x * (diff), axis = 0)
        w1 = w - LearningRate*np.reshape(gradient,(-1,1))
        Error = np.sum(np.abs(w - w1))
        w = w1
        cost = 0.5 * np.sum(np.square(diff))
        Costs.append(cost)
    print("converged after ", iter, "iterations.")
    return w, Costs

In [207]:
w, Costs = Gradien_Decent(train_x, train_y)
print("W vector:")
print(w)
from numpy import savetxt
savetxt('Costs.csv', Costs, delimiter='\n')

converged after  5859 iterations.
W vector:
[[ 0.89919933]
 [ 0.78488638]
 [ 0.84952298]
 [ 1.29780387]
 [ 0.12966251]
 [ 1.57041505]
 [ 0.99730037]
 [-0.01522461]]
